This Jupyter Notebook access the Allen Institute's Cell Type Database based on example script provided by Allen SDK (cell_types.ipynb). It then calculate the total volume of each neuron's reconstruction with the neuron_morphology package and export that data. 

In [2]:
from allensdk.core.cell_types_cache import CellTypesCache
from allensdk.core.cell_types_cache import CellTypesCache
from allensdk.api.queries.cell_types_api import CellTypesApi
from allensdk.core.cell_types_cache import ReporterStatus as RS

from neuron_morphology.features.size import total_volume
from neuron_morphology.swc_io import morphology_from_swc
from neuron_morphology.feature_extractor.data import Data
from neuron_morphology.feature_extractor.data import MorphologyLike
from neuron_morphology.feature_extractor.data import get_morphology

import pprint
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from pylmeasure import getOneMeasure

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
ctc = CellTypesCache(manifest_file='../data/cell_types/manifest.json')

In [8]:
all_features = ctc.get_all_features()

In [11]:
all_features_df = pd.DataFrame(all_features)

In [12]:
all_features_df.columns

Index(['adaptation', 'avg_isi', 'electrode_0_pa', 'f_i_curve_slope',
       'fast_trough_t_long_square', 'fast_trough_t_ramp',
       'fast_trough_t_short_square', 'fast_trough_v_long_square',
       'fast_trough_v_ramp', 'fast_trough_v_short_square', 'has_burst',
       'has_delay', 'has_pause', 'id_x', 'input_resistance_mohm', 'latency',
       'peak_t_long_square', 'peak_t_ramp', 'peak_t_short_square',
       'peak_v_long_square', 'peak_v_ramp', 'peak_v_short_square',
       'rheobase_sweep_id', 'rheobase_sweep_number', 'ri', 'sag', 'seal_gohm',
       'slow_trough_t_long_square', 'slow_trough_t_ramp',
       'slow_trough_t_short_square', 'slow_trough_v_long_square',
       'slow_trough_v_ramp', 'slow_trough_v_short_square', 'specimen_id',
       'tau', 'threshold_i_long_square', 'threshold_i_ramp',
       'threshold_i_short_square', 'threshold_t_long_square',
       'threshold_t_ramp', 'threshold_t_short_square',
       'threshold_v_long_square', 'threshold_v_ramp',
       'thresho

In [ ]:
all_features_df[["total_volume",]]

In [5]:
# human cells with full reconstructions
human_cells = ctc.get_cells(species=[CellTypesApi.HUMAN])
full_human_cells = list(filter(lambda cell: cell['reconstruction_type'] == 'full', human_cells))
# get mouse cells with full reconstructions 
mouse_cells = ctc.get_cells(species=[CellTypesApi.MOUSE])
full_mouse_cells = list(filter(lambda cell: cell['reconstruction_type'] == 'full', mouse_cells))

In [6]:
# Understanding the data structure
# len(full_human_cells)
# len(full_mouse_cells)
# pd.DataFrame(full_human_cells)
# pd.DataFrame(full_mouse_cells)
# set(pd.DataFrame(full_human_cells)['structure_area_abbrev'])
# set(pd.DataFrame(full_human_cells)['structure_layer_name'])
# set(pd.DataFrame(full_human_cells)['disease_state'])
# set(pd.DataFrame(full_mouse_cells)['structure_layer_name'])
# set(pd.DataFrame(full_mouse_cells)['structure_area_abbrev'])
# set(pd.DataFrame(full_mouse_cells)['transgenic_line'])

In [24]:
# Calculating the total volume of the neuron 
# based on the reconstruction with two different methods
df = pd.DataFrame(columns=['Species','Specimen_ID','Total_Volume'])

for i, cell in enumerate(full_mouse_cells):
    reconstruction = ctc.get_reconstruction(cell['id'])
    path = '../data/cell_types/specimen_'+str(cell['id'])+'/reconstruction.swc'
    morphology = morphology_from_swc(path)
    data = Data(morphology)
    volume = total_volume(data)
    df = df.append(
        {
            'Species':'Mouse',
            'Specimen_ID':cell['id'],
            'Total_Volume':volume,
        }, ignore_index=True)

In [25]:
# do the same for human
for i, cell in enumerate(full_human_cells):
    reconstruction = ctc.get_reconstruction(cell['id'])
    path = '../data/cell_types/specimen_'+str(cell['id'])+'/reconstruction.swc'
    morphology = morphology_from_swc(path)
    data = Data(morphology)
    volume = total_volume(data)
    df = df.append(
        {
            'Species':'Human',
            'Specimen_ID':cell['id'],
            'Total_Volume':volume,
        }, ignore_index=True)

In [28]:
# export the dataframe with total neuron volumes to a tsv file
df.to_csv('../result/total_volume.tsv',sep='\t',index=False)